# Instalação de dependências

In [8]:
# !pip install pandas
# !pip install python-dotenv
# !pip install openai
# !pip install replicate

# Importação das bibliotecas

In [9]:
# Importação das bibliotecas
import replicate
import os
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

True

# Configurações do Replicate

In [10]:
# Autenticação no Replicate
replicate_api_key = os.getenv("REPLICATE_API_KEY") # obtêm token no .env

# Salvar token cliente em variável
client = replicate.Client(api_token=replicate_api_key)

In [11]:
# Nome do modelo de IA utilizado
model_name = "meta/meta-llama-3-8b-instruct"

# Obter respostas do LLM

In [22]:
# Função que recebe o prompt e retorna a resposta do modelo
def get_chatbot_response(client, model_name, messages, temperature=0):
    # Lista das mensagens de entrada do usuário
    input_messages = []
    for message in messages:
        input_messages.append(message)
    # Converter array de mensagens em string
    input_messages = "".join(input_messages)
    # Criar o dicionário de resposta do modelo
    model_response = {
        "prompt": input_messages,  # prompt de entrada
        # número máximo de tokens (palavras ou sub-palavras) a serem gerados
        "max_new_tokens": 2_000,
        # temperatura de amostragem: nenhuma aleatoriedade (queremos resultados concretos)
        "temperature": 0.0,
        # template de prompt:
        "prompt_template": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n",
        "top_p": 0.8,  # amostragem de núcleo: 80% dos tokens mais prováveis
    }
    # Adicionar cada token à string de saída
    response = ""
    for event in client.stream(model_name,input=model_response):
        response += str(event)  # Adiciona cada token à string de saída
    # Retorna a resposta do modelo
    return response


In [24]:
# Teste da função
msg_usuario="Qual é a capital do Japão?"
resposta = get_chatbot_response(client, model_name, msg_usuario)

print(f"Usuário: {msg_usuario}\nChatBot: {resposta}")

Usuário: Qual é a capital do Japão?
ChatBot: A capital do Japão é Tóquio (em japonês,).


# Prompt Engineering

## Structured output
Armazena a resposta do chatbot em um formato estruturado (como CSV ou JSON) para poder ser utilizado em outros sistemas/aplicações (como React Native).

In [25]:
# System Prompt: Regras do Chatbot
# Definir respostas em formato estruturado
system_prompt = """
Você é um assistente útil que responde perguntas sobre capitais de países.

Seu output deve estar em um formato JSON estruturado exatamente como o abaixo. Você não tem permissão para escrever nada além do objeto JSON:
[
    {
        "country": o país do qual o usuário está perguntando a capital
        "capital": a capital do país declarada
    }
]
"""

# Teste da função com o prompt do sistema
resposta = get_chatbot_response(client, model_name, (system_prompt, "Qual é a capital da França?"))
print(resposta)

[
    {
        "country": "França",
        "capital": "Paris"
    }
]
